In [1]:
import pandas as pd
import sasoptpy as so
import os
import time

In [2]:
t0=time.time()

In [3]:
df = pd.read_csv('../data/input.csv')
df.set_index('index', inplace = True)

In [4]:
def solve_goalkeeper_problem(df, budget):
    model = so.Model(name='gk_model')
    players = df.index.tolist()
    #variables
    lineup = model.add_variables(players, name='lineup', vartype=so.binary)
    bench = model.add_variables(players, name='bench', vartype=so.binary)
    #objective
    total_xp = so.expr_sum(lineup[p] * df.loc[p, 'GW20'] for p in players) + 0.1 * so.expr_sum(bench[p] * df.loc[p, 'GW20'] for p in players)
    model.set_objective(-total_xp, name='total_xp_obj', sense='N')
    #constraints
    model.add_constraints((lineup[p] + bench[p] <=1 for p in players), name = 'lineup_or_bench')
    model.add_constraint(so.expr_sum(lineup[p] for p in players) == 1, name='single_lineup')
    model.add_constraint(so.expr_sum(bench[p] for p in players) == 1, name='single_bench')
    model.add_constraint(so.expr_sum((lineup[p] + bench[p]) * df.loc[p, 'Price'] for p in players)<= budget, name='budget_con')
    #solve step
    model.export_mps(filename='gk.mps')
    command = 'cbc gk.mps solve solu solution.txt'
    #!{command} this is with default jupyter notebook logs
    #using os library to remove all background logs
    os.system(command)
    with open('solution.txt', 'r') as f:
        for v in model.get_variables():
            v.set_value(0)
        for line in f:
            if 'objective value' in line:
                continue
            words = line.split()
            var = model.get_variable(words[1])
            var.set_value(float(words[2]))
    print("LINEUP")
    for p in players:
        if lineup[p].get_value() > 0.5:
            print(p, df.loc[p])
    print("BENCH")
    for p in players:
        if bench[p].get_value() > 0.5:
            print(p, df.loc[p])

In [5]:
#input budget here and solve for your team
solve_goalkeeper_problem(df, 9.2)

NOTE: Initialized model gk_model.
LINEUP
2 Player Name    Martinez
Price               5.2
xMins                93
GW20                4.1
Total               4.1
PPM                 0.8
Elite%             0.04
Name: 2, dtype: object
BENCH
12 Player Name    Turner
Price             3.9
xMins              90
GW20              3.3
Total             3.3
PPM              0.85
Elite%          0.178
Name: 12, dtype: object


In [6]:
t1 = time.time()
print(t1-t0, 'seconds')

1.370307207107544 seconds
